In [51]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact

In [4]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [37]:
poly_df_list = []
data_df_list = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_df_list.append(data_df)
    poly_df_list.append(poly_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_poly_df = pd.concat(poly_df_list).reset_index()

training_data_list = [total_data_df, total_poly_df]
all_df = pd.concat(training_data_list, axis=1)

# shuffle the rows
train_df = all_df.sample(frac=0.8, random_state=25)
test_df = all_df.drop(train_df.index)

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
y_labels = ["a0", "a1", "a2", "a3", "a4", "a5"]

x_dim = len(x_labels)
y_dim = len(y_labels)

x_train = train_df[x_labels]
y_train = train_df[y_labels]

x_test = test_df[x_labels]
y_test = test_df[y_labels]

In [39]:
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [49]:
model.fit(x_train, y_train, epochs=300, batch_size=10)

Epoch 1/300
490/490 [==============================] - 0s 357us/step - loss: 3.1084e-04 - accuracy: 0.7124
Epoch 2/300
490/490 [==============================] - 0s 357us/step - loss: 3.0116e-04 - accuracy: 0.7095
Epoch 3/300
490/490 [==============================] - 0s 353us/step - loss: 2.9708e-04 - accuracy: 0.7171
Epoch 4/300
490/490 [==============================] - 0s 357us/step - loss: 3.0489e-04 - accuracy: 0.7207
Epoch 5/300
490/490 [==============================] - 0s 354us/step - loss: 3.0592e-04 - accuracy: 0.7158
Epoch 6/300
490/490 [==============================] - 0s 351us/step - loss: 3.1314e-04 - accuracy: 0.7079
Epoch 7/300
490/490 [==============================] - 0s 368us/step - loss: 3.0672e-04 - accuracy: 0.7101
Epoch 8/300
490/490 [==============================] - 0s 358us/step - loss: 3.0515e-04 - accuracy: 0.7140
Epoch 9/300
490/490 [==============================] - 0s 355us/step - loss: 3.0891e-04 - accuracy: 0.7115
Epoch 10/300
490/490 [===============

Epoch 77/300
490/490 [==============================] - 0s 358us/step - loss: 2.9636e-04 - accuracy: 0.7263
Epoch 78/300
490/490 [==============================] - 0s 358us/step - loss: 3.0623e-04 - accuracy: 0.7132
Epoch 79/300
490/490 [==============================] - 0s 361us/step - loss: 3.0137e-04 - accuracy: 0.7211
Epoch 80/300
490/490 [==============================] - 0s 359us/step - loss: 2.9405e-04 - accuracy: 0.7252
Epoch 81/300
490/490 [==============================] - 0s 355us/step - loss: 3.0746e-04 - accuracy: 0.7128
Epoch 82/300
490/490 [==============================] - 0s 356us/step - loss: 3.0512e-04 - accuracy: 0.7220
Epoch 83/300
490/490 [==============================] - 0s 358us/step - loss: 2.9966e-04 - accuracy: 0.7250
Epoch 84/300
490/490 [==============================] - 0s 363us/step - loss: 2.9905e-04 - accuracy: 0.7197
Epoch 85/300
490/490 [==============================] - 0s 370us/step - loss: 3.0691e-04 - accuracy: 0.7130
Epoch 86/300
490/490 [======

490/490 [==============================] - 0s 409us/step - loss: 2.9178e-04 - accuracy: 0.7230
Epoch 153/300
490/490 [==============================] - 0s 378us/step - loss: 2.9100e-04 - accuracy: 0.7309
Epoch 154/300
490/490 [==============================] - 0s 408us/step - loss: 3.0068e-04 - accuracy: 0.7205
Epoch 155/300
490/490 [==============================] - 0s 438us/step - loss: 3.0255e-04 - accuracy: 0.7211
Epoch 156/300
490/490 [==============================] - 0s 418us/step - loss: 2.9898e-04 - accuracy: 0.7171
Epoch 157/300
490/490 [==============================] - 0s 384us/step - loss: 3.0125e-04 - accuracy: 0.7199
Epoch 158/300
490/490 [==============================] - 0s 358us/step - loss: 3.0072e-04 - accuracy: 0.7146
Epoch 159/300
490/490 [==============================] - 0s 350us/step - loss: 3.0015e-04 - accuracy: 0.7179
Epoch 160/300
490/490 [==============================] - 0s 365us/step - loss: 3.0829e-04 - accuracy: 0.7207
Epoch 161/300
490/490 [==========

490/490 [==============================] - 0s 367us/step - loss: 2.9277e-04 - accuracy: 0.7281
Epoch 228/300
490/490 [==============================] - 0s 369us/step - loss: 2.9530e-04 - accuracy: 0.7252
Epoch 229/300
490/490 [==============================] - 0s 370us/step - loss: 2.9926e-04 - accuracy: 0.7244
Epoch 230/300
490/490 [==============================] - 0s 374us/step - loss: 3.0249e-04 - accuracy: 0.7218
Epoch 231/300
490/490 [==============================] - 0s 367us/step - loss: 2.9128e-04 - accuracy: 0.7305
Epoch 232/300
490/490 [==============================] - 0s 369us/step - loss: 3.0406e-04 - accuracy: 0.7203
Epoch 233/300
490/490 [==============================] - 0s 367us/step - loss: 2.9127e-04 - accuracy: 0.7295
Epoch 234/300
490/490 [==============================] - 0s 375us/step - loss: 2.9186e-04 - accuracy: 0.7211
Epoch 235/300
490/490 [==============================] - 0s 371us/step - loss: 3.0017e-04 - accuracy: 0.7211
Epoch 236/300
490/490 [==========

In [50]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

39/39 [==============================] - 0s 378us/step - loss: 2.6915e-04 - accuracy: 0.7190
Loss: 0.00026915, Accuracy: 71.90


In [65]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    print(y_select)
    print(model.predict(x_select))
    
interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1224, 1))>